In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import polars as pl
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

# Asumiendo que es una ocupacion del 80% de camas
INDICE_OCUPACION_CAMAS = 1.25

# Agrega las columnas de poblacion de interes
COLUMNAS_POBLACION_INTERES = {str(i): i for i in range(2017, 2036)}

# Proyección de casos INT al 2035

En este cuadernillo se quiere estimar la deminada que atenderá el INT para el 2035. Con esta demanada se calcularán las cantidades de camas necesarias para suplir tal demanda. Para determinar la demanda se utilizarán:

1. Cantidad de casos teóricos por problemas de salud relevantes seleccionados: Estos se calcularon según la incidencia de cada problema de salud y la proyección de la población al 2035.

2. Cantidad de casos atendidos actualmente por cada problema de salud: Estos se obtuvieron al analizar la base de egresos hospitalarios DEIS.

Con ambos datos, se obtendrá cuanto % de atención ha cubierto el INT de los casos teóricos.

In [3]:
# Obtiene los casos de area de influencia
casos_area_de_influencia = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="casos_area_de_influencia_INT"
)

# Preprocesa el diagnostico
casos_area_de_influencia["Diagnostico"] = (
    casos_area_de_influencia["Diagnostico"].str.split(" - ").str[0]
)

# Preprocesa los diagnosticos agrupados
casos_area_de_influencia["Diagnosticos Contenidos"] = casos_area_de_influencia[
    "Diagnosticos Contenidos"
].str.split(", ")

# Renombra columnas de la poblacion
casos_area_de_influencia = casos_area_de_influencia.rename(columns=COLUMNAS_POBLACION_INTERES)

In [4]:
# Obtiene los diagnosticos relevantes a filtrar de los egresos hospitalarios
DIAGNOSTICOS_RELEVANTES = list(
    casos_area_de_influencia["Diagnostico"].str.split(" - ").str[0].unique()
)

# Obtiene los diagnosticos agrupados
DIAGNOSTICOS_AGRUPADOS = list(
    casos_area_de_influencia.explode("Diagnosticos Contenidos")["Diagnosticos Contenidos"]
    .dropna()
    .str.strip()
    .unique()
)

# Suma los diagnosticos agrupados a los diagnosticos relevantes
DIAGNOSTICOS_RELEVANTES += DIAGNOSTICOS_AGRUPADOS
DIAGNOSTICOS_RELEVANTES = sorted(DIAGNOSTICOS_RELEVANTES)

In [5]:
# Genera duplas de diagnosticos a reasignar
DIAGNOSTICOS_A_REASIGNAR = casos_area_de_influencia.query("`Diagnosticos Contenidos`.notna()")[
    ["Diagnostico", "Diagnosticos Contenidos"]
]

In [6]:
casos_area_de_influencia = casos_area_de_influencia.set_index("Diagnostico")

## Obtención de casos atendidos entre 2017 y 2020 para diagnósticos relevantes.

Estos se obtendrán del análisis de egresos hospitalarios a nivel país DEIS.

In [42]:
# Lee los egresos del Torax
egresos_torax = pl.read_csv("../data/raw/6_egresos_deis/egresos_procesados_112103.csv")

En primer lugar, se filtrarán ambas bases de datos para solamente tener la información de los diagnósticos más relevantes para el INT. Luego de esto, se calculará la cantidad de egresos, dias de estada y pacientes para cada uno de los diagnósticos. Estos insumos serán utilizados para estimar la demanda.


In [43]:
# Filtra solamente los diagnosticos mas relevantes del Torax
egresos_mas_relevantes_torax = egresos_torax.filter(
    (pl.col("DIAG1").is_in(DIAGNOSTICOS_RELEVANTES))
)

## Reasginación de diagnósticos y agrupación

En este apartado se reasignarán diagnósticos de la base de datos DEIS a un grupo mayor. Por ejemplo,
los siguientes diagnósticos:

- C341 - Tumor maligno del lóbulo superior, bronquio o pulmón
- C342 - Tumor maligno del lóbulo medio, bronquio o pulmón
- C343 - Tumor maligno del lóbulo inferior, bronquio o pulmón
- C780 - Tumor maligno secundario del pulmón
- C782 - Tumor maligno secundario de la pleura
- D381 - Tumor de comportamiento incierto o desconocido de la tráquea, de los bronquios y del pulmón

Todos estos se reasignarán al diagnóstico C34XN (N, para que no se mezclen con los C34X asignados realmente así). Esto, con el fin de poder calcular las estadísticas agrupadas para todos estos diagnósticos (egresos por persona, días de estada).

In [44]:
for row in DIAGNOSTICOS_A_REASIGNAR.itertuples():
    diagnostico_nuevo = row[1]
    diagnosticos_antiguos = row[2]

    print(f"Cambiando {diagnosticos_antiguos} a {diagnostico_nuevo}")
    egresos_mas_relevantes_torax = egresos_mas_relevantes_torax.with_columns(
        pl.col("DIAG1").replace(diagnosticos_antiguos, diagnostico_nuevo)
    )

Cambiando ['C341', 'C342', 'C343', 'C780', 'C782', 'D381'] a C34N
Cambiando ['Q201', 'Q202', 'Q203', 'Q204', 'Q205', 'Q206', 'Q208', 'Q209', 'Q210', 'Q211', 'Q212', 'Q213', 'Q214', 'Q218', 'Q220', 'Q221', 'Q222', 'Q223', 'Q224', 'Q225', 'Q228', 'Q230', 'Q231', 'Q233', 'Q240', 'Q241', 'Q244', 'Q245', 'Q246', 'Q248', 'Q249', 'Q250', 'Q251', 'Q253', 'Q254', 'Q255', 'Q256', 'Q257', 'Q258', 'Q259', 'Q264', 'Q268', 'Q272', 'Q273', 'Q288', 'Q289', 'Q311', 'Q320', 'Q321', 'Q330', 'Q331', 'Q332', 'Q334', 'Q338', 'Q341', 'Q348', 'Q676', 'Q677', 'Q678', 'Q765', 'Q766', 'Q767', 'Q768', 'Q769', 'Q780', 'Q790', 'Q798', 'Q839', 'Q850', 'Q858', 'Q859', 'Q874', 'Q893'] a QXXX


Previo a calcular las métricas para cada diagnóstico, es necesario agrupar algunos de ellos. Esto se debe a que ciertas incidencias fueron imposible de encontrar, como por ejemplo con:

- **I052 - Estenosis mitral con insuficiencia**: El 100% de los egresos y cantidad de pacientes será asignado al diagnóstico I051 - Insuficiencia mitral reumática. Sólamente se calculará la cantidad de casos teóricos para la insuficiencia mitral reumática.

- **I080 - Trastornos de las válvulas mitral y aórtica**: El 100% de los egresos y cantidad de pacientes será asignado al diagnóstico I340 - Insuficiencia (de la válvula) mitral y I350 - Estenosis (de la válvula) aórtica.

- **I081 - Trastornos de las válvulas mitral y tricúspide**: El 100% de los egresos y cantidad de pacientes será asignado al diagnóstico I340 - Insuficiencia (de la válvula) mitral y I361 - Insuficiencia no reumática (de la válvula) tricúspide.

In [45]:
# Reasigna I052
egresos_mas_relevantes_torax = egresos_mas_relevantes_torax.with_columns(
    pl.col("DIAG1").replace("I052", "I051")
)

# # Reasigna I080 a I340 y I350 (Estenosis Mitral y Aortica, respectivamente)
# egresos_mas_relevantes_torax = build_features.assign_diagnosis(
#     egresos_mas_relevantes_torax, "I080", "I340", "I350"
# )

# # Reasigna I081 a I340 y I361 (Estenosis Mitral, y Valvulopatia Tricuspide)
# egresos_mas_relevantes_torax = build_features.assign_diagnosis(
#     egresos_mas_relevantes_torax, "I081", "I340", "I361"
# )

In [47]:
# Obtiene resumen de metricas para el Torax
metricas_diags_relevantes_torax = (
    build_features.calcular_resumen_metricas_desagregadas_y_agrupadas_en_anios(
        egresos_mas_relevantes_torax, 2017, 2019
    )
)

In [51]:
metricas_diags_relevantes_torax

dias_estada_totales             n_egresos            \
                     2017  2018  2019      2017 2018 2019   
DIAG1                                                       
A152                  283   276   113         6  107   12   
B441                   93    98   415         4    2    9   
C34N                 5169  5727  5661       439  569  558   
C381                  234   253   223        15   21   19   
E848                  377   602   574        26   35   41   
I051                  183   196    85        13   14    9   
I080                  612   763   493        38   42   42   
I081                  511   981   797        29   44   42   
I232                    0    18     4         0    1    1   
I330                  698   623   492        21   22   18   
I340                  815   804   569        40   48   37   
I350                 1964  2084  1999       118  163  148   
I351                  484   737   680        37   52   49   
I352                  324   446   476        31   26   37   
I361                   34    41    32         6    6    3   
I420                  396   822   754        33   34   30   
I421                   31    78    38        12    9    5   
I495                  173   148   267        51   64   91   
I710                  379   352   483        19   24   25   
I711                  318   264    67        15   12    8   
I712                  165   139   170         8   13   15   
I715                  319   221   228        12    5   11   
J679                  864   789   969        77   70   69   
J841                 1564  1286  1799       108  108  122   
J848                  278   399   519        25   53   52   
J955                  609   225   451        53   35   44   
M051                   63    25    49         7    4    8   
M348                  290   357   421        21   29   27   
QXXX                  809   747   630       136  131  131   
Z450                  141   243   190        71  132  137   

      n_pacientes_distintos           agrupado_entre_2017_2019  \
                       2017 2018 2019                n_egresos   
DIAG1                                                            
A152                      5    4    5                      125   
B441                      4    2    8                       15   
C34N                    339  415  425                     1566   
C381                     13   18   19                       55   
E848                     20   19   24                      102   
I051                     13   14    9                       36   
I080                     37   39   39                      122   
I081                     28   40   40                      115   
I232                      0    1    1                        2   
I330                     20   21   18                       61   
I340                     38   44   36                      125   
I350                    107  145  139                      429   
I351                     34   46   47                      138   
I352                     27   26   36                       94   
I361                      6    6    3                       15   
I420                     30   30   29                       97   
I421                     11    8    5                       26   
I495                     51   63   88                      206   
I710                     19   21   25                       68   
I711                     14   11    8                       35   
I712                      8   11   14                       36   
I715                     10    5   11                       28   
J679                     63   54   57                      216   
J841                     96   96  102                      338   
J848                     23   47   46                      130   
J955                     24   20   32                      132   
M051                      5    3    6                       19   
M348        

## Estimación de casos Hospitalizados utilizando % de Hospitalización del INT

En este caso, se utilizará el % de hospitalización del INT de los pacientes ambulatorios.

In [49]:
PORCENTAJE_HOSPITALIZADOS = pd.read_excel(
    "../data/external/incidencias_y_prevalencias_INT.xlsx",
    usecols=["Diagnostico", "Porcentaje Hospitalizados de Ambulatorios"],
)

PORCENTAJE_HOSPITALIZADOS = PORCENTAJE_HOSPITALIZADOS.drop_duplicates("Diagnostico")

PORCENTAJE_HOSPITALIZADOS["Diagnostico"] = (
    PORCENTAJE_HOSPITALIZADOS["Diagnostico"].str.split(" - ").str[0]
)
PORCENTAJE_HOSPITALIZADOS = PORCENTAJE_HOSPITALIZADOS.set_index("Diagnostico")[
    "Porcentaje Hospitalizados de Ambulatorios"
]

In [52]:
casos_INT_hospitalizados_porcentaje_amb = casos_area_de_influencia[
    COLUMNAS_POBLACION_INTERES.values()
].mul(PORCENTAJE_HOSPITALIZADOS, axis=0)

# Se obtiene la cantidad de egresos que debiese tener el INT, asumiendo que trabajara con la misma
# eficiencia
egresos_estimados_INT_porcentaje_amb = casos_INT_hospitalizados_porcentaje_amb.mul(
    metricas_diags_relevantes_torax[("agrupado_entre_2017_2019", "egresos_por_paciente")], axis=0
)

# Se obtiene la cantidad de dias de estada que debiese el INT, asumiendo que trabajara con la misma
# eficiencia
dias_estada_estimados_int_porcentaje_amb = egresos_estimados_INT_porcentaje_amb.mul(
    metricas_diags_relevantes_torax[("agrupado_entre_2017_2019", "dias_estada_promedio")], axis=0
)

# Estima la cantidad de camas necesaarias por diagnostico
camas_estimadas_int_porcentaje_amb = (
    dias_estada_estimados_int_porcentaje_amb / 365.25
) * INDICE_OCUPACION_CAMAS

# Calcula las camas totales necesarias
camas_totales_int_porcentaje_amb = camas_estimadas_int_porcentaje_amb.sum()

In [53]:
print(camas_totales_int_porcentaje_amb)

2017    177.721094
2018    180.863785
2019    184.559341
2020    188.324964
2021    190.569903
2022    191.978186
2023    193.157037
2024    194.252975
2025    195.294411
2026    196.287006
2027    197.231515
2028    198.128131
2029    198.975378
2030    199.771633
2031    200.599603
2032    201.340551
2033    202.035559
2034    202.687418
2035    203.296686
dtype: float64


## Obtener resumen para MINSAL

In [60]:
resumen_metricas_hosp = metricas_diags_relevantes_torax[
    [
        ("agrupado_entre_2017_2019", "egresos_por_paciente"),
        ("agrupado_entre_2017_2019", "dias_estada_promedio"),
    ]
]

resumen_metricas_hosp.columns = [
    "egresos_por_paciente_agrupado_2017_a_2019",
    "dias_estada_promedio_agrupado_2017_a_2019",
]
resumen_casos = casos_INT_hospitalizados_porcentaje_amb[2035]
resumen_casos.name = "casos_hospitalizados_2035"

resumen_egresos = egresos_estimados_INT_porcentaje_amb[2035]
resumen_egresos.name = "egresos_2035"

resumen_dias_estada = dias_estada_estimados_int_porcentaje_amb[2035]
resumen_dias_estada.name = "dias_estada_totales_2035"

resumen_camas = camas_estimadas_int_porcentaje_amb[2035]
resumen_camas.name = "camas_totales_2035"

resumen_total_hosp = (
    resumen_metricas_hosp.merge(
        resumen_casos,
        how="left",
        left_index=True,
        right_index=True,
    )
    .merge(
        resumen_egresos,
        how="left",
        left_index=True,
        right_index=True,
    )
    .merge(
        resumen_dias_estada,
        how="left",
        left_index=True,
        right_index=True,
    )
    .merge(
        resumen_camas,
        how="left",
        left_index=True,
        right_index=True,
    )
)

ORDEN_COLUMNAS_RESUMEN = [
    "casos_hospitalizados_2035",
    "egresos_por_paciente_agrupado_2017_a_2019",
    "egresos_2035",
    "dias_estada_promedio_agrupado_2017_a_2019",
    "dias_estada_totales_2035",
    "camas_totales_2035",
]

resumen_total_hosp = resumen_total_hosp[ORDEN_COLUMNAS_RESUMEN]

Finalmente, une el resumen de casos teoricos desde el cuadernillo 1.0 con el resumen de este cuadernillo. Esto obtiene la tabla final para MINSAL.

In [61]:
resumen_casos_teoricos = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_INT"
)

resumen_casos_teoricos["Diagnostico_codigo"] = (
    resumen_casos_teoricos["Diagnostico"].str.split(" - ").str[0]
)
resumen_casos_teoricos = resumen_casos_teoricos.set_index("Diagnostico_codigo")

resumen_minsal = resumen_casos_teoricos.merge(
    resumen_total_hosp, how="right", left_index=True, right_index=True
)

resumen_minsal = resumen_minsal.reset_index(drop=True).set_index("Diagnostico")

## Guardar archivos

In [62]:
archivos_a_guardar = {
    "metricas_relevantes_INT": metricas_diags_relevantes_torax,
    "casos_hospitalizados_INT": casos_INT_hospitalizados_porcentaje_amb,
    "egresos_estimados_INT": egresos_estimados_INT_porcentaje_amb,
    "dias_estada_estimados_INT": dias_estada_estimados_int_porcentaje_amb,
    "camas_estimadas_desglosadas_INT": camas_estimadas_int_porcentaje_amb,
    "camas_totales_INT": camas_totales_int_porcentaje_amb,
    "resumen_total_hosp_INT": resumen_total_hosp,
    "resumen_MINSAL": resumen_minsal,
}

with pd.ExcelWriter("../data/interim/estimacion_atencion_cerrada_INT.xlsx") as file:
    for nombre_hoja, df_a_guardar in archivos_a_guardar.items():
        print(f"Guardando {nombre_hoja}")
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)

Guardando metricas_relevantes_INT
Guardando casos_hospitalizados_INT
Guardando egresos_estimados_INT
Guardando dias_estada_estimados_INT
Guardando camas_estimadas_desglosadas_INT
Guardando camas_totales_INT
Guardando resumen_total_hosp_INT
Guardando resumen_MINSAL
